In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from kesmarag.swpt import SWPT
from scipy import stats
from scipy.signal import impulse2
from util import calc_crest_factor
import seaborn as sns
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.kernel_ridge import KernelRidge
# max_features = 8
batch_size = 32

Using TensorFlow backend.


In [2]:
def features(id, length, dim, is_fft=False):
    kurtosis = []
    skew = []
    variance = []
    denominator = []
    maxs = []
    for j in range(1, length):
        data = pd.read_csv("train/{}/Sensor/{}.csv".format(id, j + 1)).values
        x = data[:, dim]
        # z = data[:, 1]
        # y = data[:, 2]
        if is_fft:
            x = np.fft.fft(x)
        i = 0
        while (i + 1) * 256000 < len(x) - 256000:
            k = stats.kurtosis(x[i * 256000: (i + 1) * 256000])
            kurtosis.append(k)
            s = stats.skew(x[i * 256000: (i + 1) * 256000])
            
            skew.append(s)
            v = x[i * 256000: (i + 1) * 256000].var()
            variance.append(v)

            vs = x[i * 256000: (i + 1) * 256000]
            sum = 0.0
            for v in vs:
                sum += v * v
            denominator.append(sum / len(vs))
            
            m = x[i* 256000: (i + 1) * 256000].max()
            maxs.append(m)

            i = i + 1
        k = stats.kurtosis(x[i * 256000:])
        kurtosis.append(k)
        s = stats.skew(x[i * 256000:])
        skew.append(s)
        v = x[i*256000:].var()
        variance.append(v)
        vs = x[i * 256000:]
        sum = 0.0
        for v in vs:
            sum += v * v
        denominator.append(sum / len(vs))
        m = x[i * 256000:].max()
        maxs.append(m)
        
#     data = np.array((kurtosis, skew, variance, denominator))
#     print(data.shape)
    return np.array((kurtosis, skew, variance, denominator, maxs)).T

In [3]:
data_x_1_t = features('01', 47, 0, False)
data_x_1_f = features('01', 47, 0, True)
data_1 = np.concatenate((data_x_1_t, data_x_1_f), axis=1)

data_x_2_t = features('02', 47, 0, False)
data_x_2_f = features('02', 47, 0, True)
data_2 = np.concatenate((data_x_2_t, data_x_2_f), axis=1)


In [4]:
# train_data = np.concatenate((data_1, data_2), axis=0)
train_data = data_x_1_f

In [21]:
tmp = data_x_1_t[:,0]
index = [i for i in range(len(tmp))]

In [22]:
print(tmp)

[ 1.19524717  1.3179095   1.40394873  1.46368238  1.26727321  0.79019294
  1.39096502  2.01139835  2.10317529  0.93712144  1.06287118  1.69144536
  1.52605102  1.70019456  1.59512417  2.13827669  0.8786993   0.91588916
  1.50735263  1.16137538  1.57222903  1.50424578  2.04466811  1.24912184
  1.3406418   1.69524489  1.14655496  1.84958348  1.57176052  1.63671809
  1.10607904  1.40816515  1.3645916   1.5288441   1.556423    1.25018233
  1.78803452  1.23318593  1.41797374  1.37828854  1.7281156   1.15132773
  1.31391866  1.92978944  1.25925714  1.43516356  1.6136155   2.57576283
  1.30061582  1.21248443  1.62671188  1.79486459  1.47484235  1.62142862
  2.59379703  1.60302948  1.16607471  1.77123209  1.93831466  1.79617096
  2.01308539  1.73579882  2.02350355  1.24253336  1.5826339   1.89124642
  2.232972    2.41442507  2.10918221  2.67249325  1.66778746  1.97880126
  2.33063687  2.11471376  2.61591242  2.25951006  2.77066709  1.6175397
  1.69866267  1.63980861  2.10939513  1.87823075  2.

In [5]:
print(train_data.shape)

(265, 4)


In [6]:
train_data = train_data
train_data = train_data.astype(np.float64)

C:\Users\yuanli\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: ComplexWarning: Casting complex values to real discards the imaginary part
  


In [8]:
train_label = [i for i in range(data_1.shape[0])]
# for i in range(data_2.shape[0]):
#     train_label.append(i)

In [9]:
data_x_3_t = features('03', 36, 0, False)
data_x_3_f = features('03', 36, 0, True)
# test_data = np.concatenate((data_x_3_t, data_x_3_f), axis=1)
test_data = data_x_3_f

In [10]:
test_data = test_data
test_data = test_data.astype(np.float64)
test_label = [i for i in range(test_data.shape[0])]

C:\Users\yuanli\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: ComplexWarning: Casting complex values to real discards the imaginary part
  


In [11]:
print(test_data.shape)

(208, 4)


In [12]:
svr = GridSearchCV(SVR(kernel='rbf', gamma=0.1), cv=5,
                   param_grid={"C": [1e0, 1e1, 1e2, 1e3],
                               "gamma": np.logspace(-2, 2, 5)})
svr.fit(train_data, train_label)

GridSearchCV(cv=5, error_score='raise',
       estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [1.0, 10.0, 100.0, 1000.0], 'gamma': array([  1.00000e-02,   1.00000e-01,   1.00000e+00,   1.00000e+01,
         1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [15]:
kr = GridSearchCV(KernelRidge(kernel='rbf', gamma=0.1), cv=5,
                  param_grid={"alpha": [1e0, 0.1, 1e-2, 1e-3],
                              "gamma": np.logspace(-2, 2, 5)})
kr.fit(train_data, train_label)

GridSearchCV(cv=5, error_score='raise',
       estimator=KernelRidge(alpha=1, coef0=1, degree=3, gamma=0.1, kernel='rbf',
      kernel_params=None),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': [1.0, 0.1, 0.01, 0.001], 'gamma': array([  1.00000e-02,   1.00000e-01,   1.00000e+00,   1.00000e+01,
         1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [16]:
pre = kr.predict(test_data)
print(pre)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [ ]:
max_features = max(train_data.max(), test_data.max())
print(type(max_features))

In [ ]:
model = Sequential()
model.add(Embedding(int(max_features), 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(train_data, train_label,
          batch_size=batch_size,
          epochs=15,
          validation_data=(test_data, test_label))
score, acc = model.evaluate(test_data, test_label,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)